In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from utils import *
from algo_evaluation import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df = load_data()


In [4]:
# select categories to analyze
channels = ['Wirtschaft']

filtered_df, articles_per_user, selected_users = filter_users(df, channels, min_articles=4, max_num_users=2000)
print(f"Number of selected users: {len(selected_users)}")

Number of selected users: 1478


In [5]:
# create graph
weights = iom(selected_users, articles_per_user)
threshold = np.percentile(weights.flatten(), 75)
print(f"Threshold: {threshold}")
graph = build_graph(selected_users, weights, threshold)

  0%|          | 0/1478 [00:00<?, ?it/s]

100%|██████████| 1478/1478 [00:05<00:00, 257.79it/s]


Threshold: 0.0


In [6]:

def greedy_modularity_communities(graph: nx.Graph):
    """
    Perform the greedy modularity algorithm with weights.
    """
    return list(nx.algorithms.community.greedy_modularity_communities(graph, weight='weight'))



def label_propagation_communities(graph: nx.Graph):
    """
    Perform the label propagation algorithm.
    """
    return list(nx.algorithms.community.asyn_lpa_communities(graph, weight='weight'))


def girvan_newmann_communities(graph: nx.Graph):
    """
    Perform the girvan newmann algorithm.
    """
    communities = nx.community.girvan_newman(graph)
    communities = next(communities)     # take the first split (maybe extend this)
    return list(communities)


def girvan_newman_with_weights(graph: nx.Graph):
    """
    Perform the girvan newmann algorithm with weights.
    """
    def most_valuable_edge(G):
        """Returns the edge with the highest betweenness centrality
        in the graph `G`.

        """
        betweenness = nx.edge_betweenness_centrality(G, weight='distance')
        return max(betweenness, key=betweenness.get)
    
    first_node = list(graph.nodes)[0]
    if 'distance' not in graph.nodes[first_node]:
        weights = nx.get_edge_attributes(graph, 'weight').values()
        inverse_weights = [1 / weight for weight in weights]
        nx.set_edge_attributes(graph, dict(zip(graph.edges, inverse_weights)), 'distance')

    communities = nx.community.girvan_newman(graph, most_valuable_edge=most_valuable_edge)
    communities = next(communities)     # take the first split (maybe extend this)
    return list(communities)
    

def louvain_communities(graph: nx.Graph):
    """
    Perform the louvain algorithm.
    """
    return list(nx.algorithms.community.louvain_communities(graph, weight='weight'))



algos = {
    'Greedy Modularity': greedy_modularity_communities,
    'Label Propagation': label_propagation_communities,
    'Girvan Newman': girvan_newmann_communities,
    'Girvan Newman with Weights': girvan_newman_with_weights,
    'Louvain': louvain_communities
}


In [10]:
results = perform_algos(graph, algos, True, 60 * 5)

Running Greedy Modularity...


Running Label Propagation...
Running Girvan Newman...
Algorithm Girvan Newman timed out.
Running Girvan Newman with Weights...
Algorithm Girvan Newman with Weights timed out.
Running Louvain...


In [14]:
for algo, result in results.items():
    print(f"""{algo}:
          \tTime: {result['runtime']}
          \tModularity: {result['modularity']}
          \tNumber of communities: {len(result['community']) if result['community'] else 0}""")

Greedy Modularity:
          	Time: 41.79936122894287
          	Modularity: 0.0902575818685294
          	Number of communities: 3
Label Propagation:
          	Time: 5.032747507095337
          	Modularity: -3.1101787811849135e-12
          	Number of communities: 1
Girvan Newman:
          	Time: None
          	Modularity: None
          	Number of communities: 0
Girvan Newman with Weights:
          	Time: None
          	Modularity: None
          	Number of communities: 0
Louvain:
          	Time: 14.463319063186646
          	Modularity: 0.12378368300668167
          	Number of communities: 5
